In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
import numpy as np

def chat_clean(file_path):
    '''Enter the file_path for .json Telegram Chat export'''
    with open(file_path, encoding="utf8") as f:
        d = json.load(f)
    norm_msg = json_normalize(d['messages'])
    msg_df = pd.DataFrame(norm_msg)
    msg_df_filtered = msg_df[msg_df.type=="message"]
    msg_df_filtered = msg_df_filtered[["date","text"]]
    #msg_df_filtered['text'] = msg_df_filtered['text'].str.replace('[^A-Za-z0-9]+', " ")
    msg_df_filtered = msg_df_filtered.dropna()

    return msg_df_filtered

In [2]:
!pip install folium matplotlib mapclassify
!pip install geopandas
!pip install rapidfuzz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 37.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.0 MB/s eta 0:00:00


In [3]:
import geopandas as gpd

In [4]:
from rapidfuzz.process import cdist


In [5]:
df = chat_clean("/content/drive/MyDrive/Colab Notebooks/schwarzkappler_export.json")

<ipython-input-1-22d48cf4e428>:13: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.



In [6]:
df = df.astype({"text":"string"})

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24103 entries, 33 to 24590
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    24103 non-null  object
 1   text    24103 non-null  string
dtypes: object(1), string(1)
memory usage: 564.9+ KB


In [8]:
gdf = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/wien_shp/OEFFHALTESTOGDPoint.shp")

In [9]:
gdf = gdf.astype({"HTXT":"string"})

In [10]:
from rapidfuzz import process, utils
from rapidfuzz import fuzz

#Preprocess Text Column for fuzzy string matching
text = df['text']
processed_text = [utils.default_process(tx) for tx in text]


In [11]:
#Preprocess Text Column for fuzzy string matching
htxt = gdf['HTXT']
processed_htxt = [utils.default_process(ht) for ht in htxt]

In [12]:
#Make an Array with every message as row (y-axis)
#And Score for every Public-Transport-Station (x-Axis) for every message
cdist_test = cdist(processed_text, processed_htxt, scorer=fuzz.token_set_ratio, score_cutoff=85, workers=-1)
#Return Index of Public-Transport-Sation of highest scoring element => Used for Matching the Message with Station-Name
cdist_max = np.argmax(cdist_test, axis=1)
#Create Match Index Column with result above
df["match_idx"] = cdist_max
#We are only interested in Stations found in Messages
matching_gdf = gdf.iloc[cdist_max[cdist_max != 0]]
matching_gdf['gdf_idx'] = matching_gdf.index

#We dont need all entries for one Station-Name
matching_gdf = matching_gdf.drop_duplicates()

# Reset the index of `matching_gdf` to be unique
matching_gdf = matching_gdf.reset_index(drop=True)

# Merge `df` and `matching_gdf` using the 'match_idx' column
result = pd.merge(df, matching_gdf, left_on='match_idx', right_on='gdf_idx')

# Drop the 'match_idx' column from the result if desired
#result = result.drop(columns=['match_idx'])

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
#Create Dataframe in Geopandas, which handles Geometries and Projections so we can map our results
result_gdf = gpd.GeoDataFrame(result, geometry=result.geometry, crs=4326)

In [14]:
#Export our Results to GeoPackage File Format
result_gdf.to_file("wien_kontrollen_matched.gpkg", driver="GPKG")

In [15]:
result_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 14195 entries, 0 to 14194
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   date       14195 non-null  object  
 1   text       14195 non-null  string  
 2   match_idx  14195 non-null  int64   
 3   OBJECTID   14195 non-null  float64 
 4   HTXT       14195 non-null  string  
 5   HTXTK      14195 non-null  object  
 6   HLINIEN    14195 non-null  object  
 7   DIVA_ID    0 non-null      float64 
 8   LTYP       14195 non-null  float64 
 9   WEBLINK1   14195 non-null  object  
 10  geometry   14195 non-null  geometry
 11  gdf_idx    14195 non-null  int64   
dtypes: float64(3), geometry(1), int64(2), object(4), string(2)
memory usage: 1.4+ MB


In [16]:
from datetime import datetime
#Rework our GeoDataframe Time-Information in order to make animations
result_gdf["dt_obj"] = result_gdf["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))

In [17]:
#Rework our GeoDataframe Time-Information in order to make animations
result_gdf["hour"] = result_gdf["dt_obj"].apply(lambda x : x.hour)

In [18]:
# Group by 'hour' and 'geometry' and count the number of elements per group
grouped = result_gdf.groupby(["hour", "geometry", "HTXT"]).size()

# Rename the count column to 'count'
grouped = grouped.rename('count')

# Convert the resulting Series back to a DataFrame
grouped = grouped.to_frame().reset_index()

In [19]:
import plotly.express as px

In [20]:
grouped_gdf = gpd.GeoDataFrame(grouped, geometry=grouped.geometry, crs=4326)

In [21]:
grouped_gdf = grouped_gdf.to_crs(4326)
grouped_gdf['lon'] = grouped_gdf.centroid.x 
grouped_gdf['lat'] = grouped_gdf.centroid.y

<ipython-input-21-cd56fac25aa7>:2: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


<ipython-input-21-cd56fac25aa7>:3: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [22]:
grouped_gdf

,hour,geometry,HTXT,count,lon,lat
0,0,POINT (16.34224 48.18829),Margaretengürtel,1,16.342242,48.188290
1,0,POINT (16.33199 48.17437),Meidling,1,16.331992,48.174367
2,0,POINT (16.26107 48.19741),Hütteldorf,1,16.261070,48.197414
3,0,POINT (16.38403 48.24111),Handelskai,2,16.384031,48.241109
4,0,POINT (16.33765 48.19666),Westbahnhof,1,16.337651,48.196656
...,...,...,...,...,...,...
2301,23,POINT (16.39233 48.21854),Praterstern,1,16.392332,48.218541
2302,23,POINT (16.45063 48.27698),Leopoldau,2,16.450632,48.276975
2303,23,POINT (16.40030 48.25682),Floridsdorf,1,16.400300,48.256821
2304,23,POINT (16.42001 48.17019),Simmering,1,16.420009,48.170186


In [23]:
np.mean(grouped_gdf["lon"])

16.367726467740667

In [24]:
%matplotlib inline

In [25]:
fig = px.density_mapbox(grouped_gdf, lat='lat', lon='lon', z='count', radius=15,
                        center=dict(lat=np.mean(grouped_gdf["lat"]), lon=np.mean(grouped_gdf["lon"])), zoom=11,
                        mapbox_style="stamen-terrain")
fig.show()

In [26]:
fig = px.density_mapbox(grouped_gdf, lat='lat', lon='lon', z='count', radius=15, hover_data=["hour", "HTXT"],
                        center=dict(lat=np.mean(grouped_gdf["lat"]), lon=np.mean(grouped_gdf["lon"])), zoom=11,
                        mapbox_style="stamen-terrain")
fig.show()

In [27]:
import plotly.express as px

fig = px.scatter(grouped_gdf, x="lon", y="lat", animation_frame="hour", animation_group="HTXT",
           size="count",  hover_name="HTXT",
           log_x=True, size_max=55, range_x=[np.min(grouped_gdf["lon"]),np.max(grouped_gdf["lon"])], 
                                    range_y=[np.min(grouped_gdf["lat"]),np.max(grouped_gdf["lat"])])

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()

In [28]:
import plotly.express as px

fig = px.scatter_mapbox(grouped_gdf, lat='lat', lon='lon',  hover_data=["hour", "HTXT"],
                        center=dict(lat=np.mean(grouped_gdf["lat"]), lon=np.mean(grouped_gdf["lon"])), zoom=11,
                        mapbox_style="stamen-terrain", animation_frame="hour", animation_group="HTXT",
           size="count",  hover_name="HTXT")

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()

In [29]:
fig = px.density_mapbox(grouped_gdf, lat='lat', lon='lon', z='count', radius=20, hover_data=["hour", "HTXT"], animation_frame="hour", animation_group="HTXT",
                        center=dict(lat=np.mean(grouped_gdf["lat"]), lon=np.mean(grouped_gdf["lon"])), zoom=10.5,
                        mapbox_style="stamen-terrain")
fig["layout"].pop("updatemenus") # optional, drop animation buttons

fig.show()

In [30]:
result.iloc[10977]["text"]

'Auch bei rennweg 2 am bahnsteig'

In [31]:
from rapidfuzz import fuzz

# create example DataFrame


# create example string list
string_list = ['hello', 'world', 'foo', 'bar']

# define function to apply to each element of match_list
@np.vectorize
def my_function(x):
    # use fuzz.ratio to check the similarity between x and each string in string_list
    ratios = [fuzz.ratio(x, s) for s in processed_htxt]
    max_ratio = max(ratios)
    # set result based on maximum ratio
    if max_ratio >= 80:
        return 'match found'
    else:
        return 'no match'

# apply function to match_list column and insert result into new column
df['result'] = my_function(df['text'])

# print resulting DataFrame
print(df)

                      date                                               text  \
33     2019-11-13T12:22:07                               Kontrolle Karlsplatz   
55     2019-11-13T13:12:55                    schau ma was sich mehr auszahlt   
70     2019-11-13T13:58:06  U2 -Station Schottentor; zivil, kurze graue Ha...   
117    2019-11-14T08:14:38                                       Volkstheater   
122    2019-11-14T09:34:19                                         Landstraße   
...                    ...                                                ...   
24586  2023-06-02T19:28:08                                          Auch Lift   
24587  2023-06-02T19:39:46  U1 Karlsplatz und Oper Ausgang Kontrolle stati...   
24588  2023-06-02T19:51:16                                       Still active   
24589  2023-06-02T19:58:07                                            Ongoing   
24590  2023-06-02T20:09:39                                  Oper Ausgang frei   

       match_idx       resu

In [32]:
test_df = df.iloc[0:50]
test_text = test_df['text']
test_processed_text = [utils.default_process(tx) for tx in test_text]

In [33]:
for (i, processed_query) in enumerate(test_processed_text):
  match_htxt = process.extractOne(processed_query, processed_htxt, processor=None, score_cutoff=87)
  print(match_htxt)


('karlsplatz', 90.0, 1401)
None
('schottentor', 90.0, 1325)
('volkstheater', 100.0, 1575)
('cumberlandstraße', 90.0, 126)
('meidling', 90.0, 1355)
None
('friedrich engels platz', 90.0, 3333)
('spittelau', 90.0, 1361)
None
('stadion', 90.0, 1415)
None
('seestadt', 90.0, 1345)
('hauptbahnhof', 90.0, 1567)
None
('schottenring', 95.0, 1324)
None
('vorgartenstraße', 90.0, 1576)
('westbahnhof', 90.0, 1305)
None
None
('zieglergasse', 90.0, 1311)
('karlsplatz', 90.0, 1401)
None
None
None
('floridsdorf', 90.0, 1430)
None
None
('volkstheater', 90.0, 1575)
None
('volkstheater', 90.0, 1575)
None
None
('kahlenberg', 90.0, 808)
None
None
('krausegasse', 90.0, 3182)
('schottenring', 90.0, 1324)
('gasometer', 90.0, 554)
('schottenring', 90.0, 1324)
None
('reumannplatz', 90.0, 1364)
None
None
None
None
None
None
None


In [34]:
for (i, processed_query) in enumerate(test_processed_text):
  match_htxt = process.extract(processed_query, processed_htxt, scorer=fuzz.WRatio, limit=5, score_cutoff=85)
  print(match_htxt)
  print(processed_query)
  print("---------------------------------------------")
  print("---------------------------------------------")

[('karlsplatz', 90.0, 1401), ('karlsplatz', 90.0, 3712), ('karlsplatz', 90.0, 3713), ('karlsplatz', 90.0, 3714), ('karlsplatz', 90.0, 3715)]
kontrolle karlsplatz
---------------------------------------------
---------------------------------------------
[]
schau ma was sich mehr auszahlt
---------------------------------------------
---------------------------------------------
[('schottentor', 90.0, 1325), ('schottentor', 90.0, 5173), ('schottentor', 90.0, 5174), ('schottentor', 90.0, 5175), ('schottentor', 90.0, 5176)]
u2  station schottentor  zivil  kurze graue haare schwarze lederjacke
---------------------------------------------
---------------------------------------------
[('volkstheater', 100.0, 1575), ('volkstheater', 100.0, 4852), ('volkstheater', 100.0, 4853), ('volkstheater', 100.0, 4854), ('volkstheater', 100.0, 4855)]
volkstheater
---------------------------------------------
---------------------------------------------
[('cumberlandstraße', 90.0, 126), ('cumberlandstra

In [ ]:
from rapidfuzz.process import cdist
cdist_test = cdist(processed_text, processed_htxt, scorer=fuzz.token_set_ratio, score_cutoff=85, workers=-1)

In [ ]:
duplicates_list = []

for distances in cdist_test:
    # Get indices of duplicates
    indices = np.argwhere(~np.isin(distances, [100, 0])).flatten()
    # Get names from indices
    names = list(map(processed_htxt.__getitem__, indices))
    duplicates_list.append(names)

In [ ]:
df["match_idx"] = cdist_max

In [ ]:
matching_gdf = gdf.iloc[cdist_max[cdist_max != 0]]

In [ ]:
#matching_gdf = gdf.iloc[cdist_max[cdist_max != 0]]
matching_gdf = gdf.iloc[cdist_max[cdist_max != 0]]
matching_gdf['gdf_idx'] = matching_gdf.index
matching_gdf = matching_gdf.drop_duplicates()

# Reset the index of `matching_gdf` to be unique
matching_gdf = matching_gdf.reset_index(drop=True)

# Merge `df` and `matching_gdf` using the 'match_idx' column
result = pd.merge(df, matching_gdf, left_on='match_idx', right_on='gdf_idx')

# Drop the 'match_idx' column from the result if desired
#result = result.drop(columns=['match_idx'])

In [ ]:
result.info()

In [ ]:
gdf_result = gpd.GeoDataFrame(result, crs="EPSG:4326", geometry=result.geometry)

In [ ]:
!pip install geoplot

In [ ]:
# Import geoplot
import geoplot
import geoplot.crs as gcrs

# Plot heatmap
ax = geoplot.kdeplot(gdf_result) #, projection=gcrs.AlbersEqualArea())

# Add polygons
geoplot.polyplot(gdf_result, ax=ax)

In [ ]:
df.head(100)

In [ ]:
df

In [ ]:
result

In [ ]:
df

In [ ]:
methods = []
method_score = []
for method in dir(fuzz):
  if method[0] == "_":
    pass
  elif method == "partial_ratio_alignment":
    pass
  elif method[0] != "a":
    print(method)
    cdist_test = cdist( test_processed_text, processed_htxt, scorer=eval("fuzz."+method), score_cutoff=80 ) 
    #, workers=-1)
    unique = np.unique(cdist_test, return_counts=True)
    methods.append(method)
    method_score.append(sum(unique[0]*unique[1]))
# unique = np.unique(cdist_test[cdist_test>75], return_counts=True)

In [ ]:
method_score

In [ ]:
methods

In [ ]:
for (i, processed_query) in enumerate(test_processed_text):
  match_htxt = process.extract(processed_query, processed_htxt, scorer=fuzz.token_set_ratio, limit=5, score_cutoff=85)
  print(match_htxt)
  print(processed_query)
  print("---------------------------------------------")
  print("---------------------------------------------")

In [ ]:
res = [i for i in range(len(processed_htxt)) if processed_htxt[i] == None]


In [ ]:
res

In [ ]:
for method in dir(fuzz):
  if method[0] == "_":
    pass
  elif method[0] != "a":
    print(eval("fuzz."+method))

In [ ]:
np.unique(cdist_test[cdist_test>75], return_counts=True)

In [ ]:
df["text"].str.lowercase().contains("richtung")

In [ ]:
!pip install fuzzywuzzy
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
def fuzzy_merge(df_1, df_2, key1, key2, threshold=95, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].astype(str).tolist()
    
    m = df_1[key1].astype(str).apply(lambda x: process.extract(x, s, limit=limit, scorer=fuzz.partial_ratio))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    
    
    df_1['matches'] = m2
    
    return df_1

In [ ]:
for i in range(19500, df.index.max() + 1500, 1500):
  if i > df.index.max():
    i = df.index.max()
    fuzzy_merge(df.loc[i-1500:i], gdf, "text", "HTXT", 80, limit=1).to_csv("/content/drive/MyDrive/Colab Notebooks/test{}_{}.csv".format(str(i-1500), str(i)))
  elif i == 0:
    fuzzy_merge(df.loc[0:i], gdf, "text", "HTXT", 80, limit=1).to_csv("/content/drive/MyDrive/Colab Notebooks/test{}_{}.csv".format(str(0), str(i)))
  elif i > 0:
    fuzzy_merge(df.loc[i-1500:i], gdf, "text", "HTXT", 80, limit=1).to_csv("/content/drive/MyDrive/Colab Notebooks/test{}_{}.csv".format(str(i-1500), str(i)))
  else:
    print(i)

In [ ]:
#TODO Station/Stationär wird zu Stadion gemapped
#TODO Richtung/Gen Endstation -> Endstationen in extra spalte Flaggen und Wenn Endstation -> Check nach Richtung/Gen in Nachricht, dann rausnehmen
#TODO Meistgefundene Stationen ausgeben und schauen ob hier auch systematisch fehlerhafte Matches vorkommen

In [ ]:
for i in range(4500, df.index.max() + 1500, 1500):
  print(i-1500, i)


In [ ]:
for i in range(0, df.index.max() + 1500, 1500):
    print(df.loc[i-1500:i])

In [ ]:
for i in range(0, df.index.max() + 1500, 1500):
  if i > df.index.max():
    i = df.index.max()
    fuzzy_merge(df.loc[i-1500:i], gdf, "text", "HTXT", 80, limit=1).to_csv("/content/drive/MyDrive/Colab Notebooks/test{}_{}.csv".format(str(i-1500), str(i)))
  elif i == 0:
    fuzzy_merge(df.loc[0:i], gdf, "text", "HTXT", 80, limit=1).to_csv("/content/drive/MyDrive/Colab Notebooks/test{}_{}.csv".format(str(0), str(i)))
  elif i > 0:
    fuzzy_merge(df.loc[i-1500:i], gdf, "text", "HTXT", 80, limit=1).to_csv("/content/drive/MyDrive/Colab Notebooks/test{}_{}.csv".format(str(i-1500), str(i)))
  else:
    print(i)
  
  


In [ ]:
df_fuz = fuzzy_merge(df, gdf, "text", "HTXT", 80, limit=1).to_csv("/content/drive/MyDrive/Colab Notebooks/test2.csv")


In [ ]:



20000/50

In [ ]:
def extract_locs(classified_dict):
  locs = []
  text = ""
  for tc in classified_dict:
    try:
      if list(tc.values())[2] == (i + 1):
        text = text + list(tc.values())[3].replace("#", "")
      elif list(tc.values())[2] != (i + 1):
        locs.append(text)
        text = list(tc.values())[3]
    except NameError:
      text = list(tc.values())[3]

      pass
    i = list(tc.values())[2]
  return locs



In [ ]:
locs = extract_locs(test_class)

In [ ]:
extract_locs(test_class)

In [ ]:
for tc in test_class:
  print(tc.values())

In [ ]:
!pip install transformers
from transformers import pipeline

In [ ]:
classifier = pipeline('ner', model="fhswf/bert_de_ner")

In [ ]:
test_result = classifier(df_test.loc[320].text)

In [ ]:
def dict_to_str(in_dict):
  location = ""
  for res in in_dict:
    location = location + list(res.values())[3].replace("#", "")
  return location


In [ ]:
df_test = df_test.drop(columns="location")
df_test["location"] = df_test.text.apply(lambda x: extract_locs(classifier(x)))

In [ ]:
df_test

In [ ]:
gdf

In [ ]:
df2 = pd.read_json("/content/drive/MyDrive/Colab Notebooks/schwarzkappler.json")

In [ ]:
df2 = pd.io.json.json_normalize(df2.messages)
df2 = pd.io.json.json_normalize(df2.text_entities)

In [ ]:
df2.loc[~df2[0].isnull()]